# Import libraries

In [ ]:
%matplotlib notebook
import warnings
warnings.filterwarnings('ignore')
import unittest
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
from matplotlib import rc
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import sklearn 
from sklearn import linear_model
import statsmodels.api as sm
import requests 
import json
from operator import itemgetter
from multiprocessing import Pool
from slugify import slugify

# Data acquisition

## Web scraping

In [ ]:
html_path=""
res = requests.get(html_path) 
html_doc =  res.text
doc = BeautifulSoup(html_doc, "html.parser")

doc_class1 = soup.find_all("div", class_= "class1", limit=1)
doc_class1_spans = doc_class1[0].find_all("span", limit=2)



## Csv, Excel files

In [ ]:
# Csv
pd.read_csv("","")
# Excel
pd.read_excel(file, sheet_name='', skiprows=n)

# Text files 
file = open(file_name, "r")
token = file.readline()
file.close()

## API  

In [ ]:
def get_json_from_link(url, my_token):
    token_header = {'Authorization' : 'token ' + my_token}
    req = requests.get(url, headers=token_header)
    json_res = json.loads(req.text)
    return json_res



# Exploring data

## Data 1

In [ ]:
# Clean data

# Drop columns
df = df.drop(columns=['', ''])

# Purge empty values
df = df.loc[df[''] != 'nc']
df = df.loc[df[''] != 0]

# Cast to float
df[''] = df[''].astype(str).astype(float)

# Rename columns
df_new = df.rename(index=str, columns={"Origin col1": "COL1_NEW", "Origin col2": "COL2_NEW"})

# Regex
reg = r"([\D]*)(\d+)(.*),([\w\s]*)"
string = "PARACETAMOL ZYDUS 500 mg, gélule"
match = re.search(reg, string)
df = df["denomination"].str.extract(reg)


In [ ]:
#Plot histogram, correlation, reclean data


In [ ]:
# Scatter Matrix
col = ['','','']

grid = sns.pairplot(df[col], size=3)
grid = grid.map_upper(plt.scatter)
grid = grid.map_diag(plt.hist, bins = 10, edgecolor = 'k')
grid = grid.map_lower(plt.scatter)

## Data 2 

In [ ]:
# Take only departments which are starting with digits
filter = df['DEPARTEMENT'].str.contains(r'[\d]')
df = df[filter]



## Data 3 

In [ ]:
# Sort by department and depassement
df = df.sort_values(by=['DEPARTEMENT', 'DEPASSEMENTS'],  ascending=[True, False])


## Merging data 

In [ ]:
# Merge
df1 = df1.merge(df2, on='DEPARTEMENT')


# Explore merged data

In [ ]:
# Group by sample
# Ratio de dépassement d'honoraires par densité de médecin par spécialité et par territoire
col = ['RATIO_DEPASSEMENTS_EFFECTIFS']
df_new = df.groupby(['SPECIALITE','DEPARTEMENT'])[col].min()



# Add new features from the merged data

# Some models based on the data

# Direct answer to the questions thanks to the models

## Question 1 

## Question 2 

## Question 3

# Conclusion